<a href="https://colab.research.google.com/github/aman3013/Fine-tuning-Amharic-NER/blob/Task-3/Finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tokenizers


In [ ]:
!pip install pytesseract pdf2image

In [ ]:
!pip install amseg

In [4]:
# Import Libraries
import pandas as pd
from amseg.amharicSegmenter import AmharicSegmenter
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import load_from_disk
from datasets import load_dataset


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import pandas as pd

# Define the file path
file_path = '/content/drive/My Drive/merged_file.conll'

# Initialize lists to hold the data
data = []
current_sentence = []

# Read the file and process it
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading and trailing whitespace
        if line:  # If the line is not empty
            parts = line.split()
            if len(parts) == 2:  # Ensure it has exactly two parts (word, label)
                current_sentence.append(parts)  # Append the (word, label) pair
        else:
            if current_sentence:  # If there's a current sentence to save
                data.append(current_sentence)  # Append the sentence to the data
                current_sentence = []  # Reset for the next sentence

# Handle the last sentence if there's no trailing newline
if current_sentence:
    data.append(current_sentence)

# Create a DataFrame from the data
df = pd.DataFrame([(word, label) for sentence in data for word, label in sentence], columns=['Word', 'Label'])

# Display the DataFrame
print(df.head())


   Word Label
0   ይህን     O
1   መፍጫ     O
2  ከሁሉም     O
3  የተሻለ     O
4    ሆኖ     O


In [7]:
from amseg.amharicSegmenter import AmharicSegmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [8]:
# Load the Model's Tokenizer
model_name = "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/404 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Separate tokens and labels
tokens = df['Word'].tolist()  # Extract tokens
labels = df['Label'].tolist()  # Extract labels

In [10]:
# Print the separated tokens and labels
print("Tokens:", tokens)
print("Labels:", labels)

Tokens: ['ይህን', 'መፍጫ', 'ከሁሉም', 'የተሻለ', 'ሆኖ', 'አግኝተነዋል', '1', 'አስተማማኝ', 'የሆነ', 'ዕቃ', 'በኤሌክትሪክ', 'የሚሰራ', 'የሽንኩርት', 'የስጋ', 'የጨጓራ', 'መፍጫ', 'በ', '15', 'ሰከንዶች', 'ዉስጥ', 'ድቅቅ', 'አድርጎ', 'የሚፈጭ', 'ምላጭ', 'በ', '3', 'ሊትር', 'እና', '5ሊትር', 'ፈጣን', 'የሆኑ', '3ሊትር', '2100', '5ሊትር', '2600', 'የ1', 'አመት', 'ዋስትና', 'ያላቸዉ', 'በጣም', 'የሚያስደስት', 'ዕቃ', 'ደዉለዉ', 'ማዘዝ', 'ይችላሉ', 'ይህን', 'ዕቃ', 'እና', 'ሌላ', '1', 'እቃ', 'ጨምረዉ', 'ሲገዙ', '1', 'ምርጥ', 'ቢላ', 'በነፃ', 'አድራሻ', 'ቁጥር', '1', 'ልደታ', 'ወደ', 'ባልቻ', 'ሆስታል', 'ገባ', 'ብሎ', 'አህመድ', 'ህንፃ', 'ላይ', '1ኛፎቅ', '105', 'ቁጥር', '2', '22', 'አውራሪስ', 'ሆቴል', 'አጠገብ', 'በፀጋ', 'ህንፃ', '3ኛ', 'ፎ', 'ቅ', 'ከ', 'ሊፍቱ', 'በ', 'ግራ', 'የሱቅ', 'ቁጥር', '10', 'ይደዉሉልን', 'ባሉበት', 'ያለተጨማሪ', 'ክፍያ', 'ማዘዝ', 'ይችላሉ', 'ክፍያዎንበሞባይልባንኪንግመፈፀምምይችላሉ', 'በተጨማሪ', 'ከ1000ብር', 'በላይ', 'የሆኑ', 'ሁለትዕቃዎች', 'ሲገዙ', 'ስጦታ', 'እንልክለዎታለን', '/', 'ቻናላችንን', 'ለጓደኛዎ', 'ሸር', 'ማድረግዎን', 'አይርሱ', '0933334444', '0944109295', '0946242424', 'ማጂክ', 'መወልወያ', 'ውሃ', 'በከፍተኛ', 'ደረጃ', 'ይመጣል', 'በራሱ', 'ይጨምቃል', 'ከእጅ', 'ንኪኪ', 'ነፃ', 'ዋጋ', '900', 'ብር', 'ከነፃ', 'ዲሊቨሪ', 'ጋር', 'አድራሻ'

In [11]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(segmenter, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = segmenter.amharic_tokenizer(word)  # Use the correct tokenizer method
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels


In [12]:
# Step 5: Align Tokens and Labels
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [14]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[100:150], new_labels[100:150]):
    print(f"{token}: {label}")

ሁለትዕቃዎች: O
ሲገዙ: O
ስጦታ: O
እንልክለዎታለን: O
/: O
ቻናላችንን: O
ለጓደኛዎ: O
ሸር: O
ማድረግዎን: O
አይርሱ: O
0933334444: O
0944109295: O
0946242424: O
ማጂክ: O
መወልወያ: O
ውሃ: O
በከፍተኛ: O
ደረጃ: O
ይመጣል: O
በራሱ: O
ይጨምቃል: O
ከእጅ: O
ንኪኪ: O
ነፃ: O
ዋጋ: B-PRICE
900: I-PRICE
ብር: I-PRICE
ከነፃ: O
ዲሊቨሪ: O
ጋር: O
አድራሻ: O
ቁጥር: O
1: O
ልደታ: B-LOC
ወደ: O
ባልቻ: B-LOC
ሆስታል: O
ገባ: O
ብሎ: O
አህመድ: O
ህንፃ: O
ላይ: O
1ኛፎቅ: O
105: O
ቁጥር: O
2: O
22: O
አውራሪስ: O
ሆቴል: O
አጠገብ: O


In [15]:
num_token = len(new_tokens)
print("Number of tokens:", num_token)

Number of tokens: 130656


In [16]:
#function to split the big list into smaller chunks (examples)
def split_into_examples(tokens, labels, max_length=512):
    examples_tokens = []
    examples_labels = []
    current_tokens = []
    current_labels = []

    for token, label in zip(tokens, labels):
        if len(current_tokens) >= max_length:
            examples_tokens.append(current_tokens)
            examples_labels.append(current_labels)
            current_tokens = []
            current_labels = []

        current_tokens.append(token)
        current_labels.append(label)

    # Add the last example if it's not empty
    if current_tokens:
        examples_tokens.append(current_tokens)
        examples_labels.append(current_labels)

    return examples_tokens, examples_labels

In [17]:
# use the above function to split our big list into examples:
max_example_length = 512  # You can adjust this value
tokens_list, labels_list = split_into_examples(tokens, labels, max_example_length)

In [18]:
len(labels_list)

255

In [19]:
# process these examples:

all_input_ids = []
all_attention_masks = []
all_labels = []

for tokens, labels in zip(tokens_list, labels_list):
    aligned_tokens, aligned_labels = align_tokens_with_labels(segmenter, tokens, labels)

    encoding = tokenizer(
        aligned_tokens,
        is_split_into_words=True,
        padding='max_length',
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'][0]
    attention_mask = encoding['attention_mask'][0]

    label_ids = [-100] * 256
    for i, label in enumerate(aligned_labels):
        if i < 256:
            label_ids[i] = label

    all_input_ids.append(input_ids.tolist())
    all_attention_masks.append(attention_mask.tolist())
    all_labels.append(label_ids)

In [20]:
# Check the first example of each processed list
print("Input IDs (first example):", all_input_ids[0])
print("Attention Mask (first example):", all_attention_masks[0])
print("Labels (first example):", all_labels[0])

Input IDs (first example): [0, 34142, 2370, 4722, 15741, 1464, 101748, 157154, 40327, 189310, 4413, 4974, 34690, 106, 91081, 4236, 4236, 8822, 43580, 35587, 6550, 728, 37830, 10824, 110020, 146329, 2627, 31415, 623, 49774, 9878, 101290, 64590, 4984, 623, 16333, 41390, 3851, 2370, 4722, 15741, 728, 423, 12560, 148948, 60688, 186706, 60757, 5653, 5653, 88846, 2627, 6980, 13799, 6, 136334, 13799, 728, 138, 10085, 116946, 2302, 190, 12354, 116946, 242766, 105574, 138, 12354, 116946, 142483, 190, 12354, 116946, 168354, 623, 418, 98040, 227536, 135860, 88603, 36059, 54407, 4555, 28168, 35587, 6550, 21368, 12131, 176434, 4363, 9039, 7872, 94164, 34142, 35587, 6550, 2302, 44020, 106, 4708, 6550, 6, 200762, 3376, 12131, 6021, 5430, 47797, 106, 168220, 18521, 4712, 111554, 29307, 140042, 61264, 106, 19730, 4555, 3348, 5156, 93076, 17231, 62922, 46945, 2202, 6, 21080, 27231, 115798, 6, 49334, 29307, 2269, 106, 9171, 31531, 5653, 21213, 61264, 116, 1039, 45172, 3851, 7423, 2095, 62922, 17680, 2202

In [21]:
# Convert labels to strings
all_labels = [[str(label) for label in example_labels] for example_labels in all_labels]


In [22]:
# Ensure all data is in the correct format
tokenized_data = {
    "input_ids": [[int(id) for id in example] for example in all_input_ids],
    "attention_mask": [[int(mask) for mask in example] for example in all_attention_masks],
    "labels": all_labels,
}

In [23]:
# Create the dataset
dataset = Dataset.from_dict(tokenized_data)

In [24]:
# Check the dataset structure
print("Dataset structure:", dataset)

Dataset structure: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 255
})


In [25]:
# Create label mapping
unique_labels = set(label for example_labels in all_labels for label in example_labels if label != '-100')
label_map = {label: i for i, label in enumerate(unique_labels)}

In [26]:
# Convert labels back to integers
def convert_labels_to_ids(example):
    example['labels'] = [label_map[label] if label != '-100' else -100 for label in example['labels']]
    return example

dataset = dataset.map(convert_labels_to_ids)

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

In [27]:
print(dataset[0])  # Print the first example in the dataset

{'input_ids': [0, 34142, 2370, 4722, 15741, 1464, 101748, 157154, 40327, 189310, 4413, 4974, 34690, 106, 91081, 4236, 4236, 8822, 43580, 35587, 6550, 728, 37830, 10824, 110020, 146329, 2627, 31415, 623, 49774, 9878, 101290, 64590, 4984, 623, 16333, 41390, 3851, 2370, 4722, 15741, 728, 423, 12560, 148948, 60688, 186706, 60757, 5653, 5653, 88846, 2627, 6980, 13799, 6, 136334, 13799, 728, 138, 10085, 116946, 2302, 190, 12354, 116946, 242766, 105574, 138, 12354, 116946, 142483, 190, 12354, 116946, 168354, 623, 418, 98040, 227536, 135860, 88603, 36059, 54407, 4555, 28168, 35587, 6550, 21368, 12131, 176434, 4363, 9039, 7872, 94164, 34142, 35587, 6550, 2302, 44020, 106, 4708, 6550, 6, 200762, 3376, 12131, 6021, 5430, 47797, 106, 168220, 18521, 4712, 111554, 29307, 140042, 61264, 106, 19730, 4555, 3348, 5156, 93076, 17231, 62922, 46945, 2202, 6, 21080, 27231, 115798, 6, 49334, 29307, 2269, 106, 9171, 31531, 5653, 21213, 61264, 116, 1039, 45172, 3851, 7423, 2095, 62922, 17680, 2202, 2350, 6839,

In [28]:
# Split the dataset
train_test_split_ratio = 0.8
train_dataset, test_dataset = dataset.train_test_split(test_size=1 - train_test_split_ratio).values()


In [29]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 204
})

In [30]:
# Further split the training dataset into training and validation set
train_val_split_ratio = 0.8
train_dataset, eval_dataset = train_dataset.train_test_split(test_size=1 - train_val_split_ratio).values()

In [31]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 41
})

In [ ]:
# Initialize the model
num_labels = len(label_map)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

In [ ]:
!swapoff /swapfile
!dd if=/dev/zero of=/swapfile bs=1M count=1024
!mkswap /swapfile
!swapon /swapfile

In [ ]:
!pip install --upgrade accelerate

In [ ]:
!pip install evaluate

In [36]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Reduce batch size
    per_device_eval_batch_size=2,   # Reduce evaluation batch size
    gradient_accumulation_steps=8,  # Accumulate gradients to simulate larger batches
    num_train_epochs=3,
    fp16=True,  # Keep mixed precision enabled
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
)

In [37]:
import numpy as np
import evaluate

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [38]:
# Compute Metrics
def compute_metrics(pred):
    # Extract the predictions and labels
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)  # Take the argmax to get the predicted class

    # Flatten the predictions and labels to ignore padding (-100)
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        # Only add non-ignored labels (label != -100)
        for pred_token, label_token in zip(prediction, label):
            if label_token != -100:
                true_predictions.append(pred_token)
                true_labels.append(label_token)

    # Compute the metrics
    accuracy = accuracy_metric.compute(predictions=true_predictions, references=true_labels)
    f1 = f1_metric.compute(predictions=true_predictions, references=true_labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

In [39]:
# Reinitialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=None,  # or your custom tokenizer
    compute_metrics=compute_metrics  # Pass the custom metrics function
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [40]:
# Start Training
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.788911,0.712176,0.690764
1,No log,0.736849,0.711414,0.689733
2,No log,0.724654,0.714844,0.693291


TrainOutput(global_step=30, training_loss=0.880499521891276, metrics={'train_runtime': 325.9026, 'train_samples_per_second': 1.5, 'train_steps_per_second': 0.092, 'total_flos': 221966991624192.0, 'train_loss': 0.880499521891276, 'epoch': 2.926829268292683})

In [41]:
# Evaluate the model on the test set
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 0.752111554145813, 'eval_accuracy': 0.6965379901960784, 'eval_f1': 0.6769658248616345, 'eval_runtime': 1.5685, 'eval_samples_per_second': 32.516, 'eval_steps_per_second': 16.577, 'epoch': 2.926829268292683}


In [44]:
save_path = '/content/drive/My Drive/fine_tuned_model'
import os
os.makedirs(save_path, exist_ok=True)
model.save_pretrained(save_path)